<a href="https://colab.research.google.com/github/saurabhsingh1411/Deep_Learning_Projects/blob/main/NLP/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/

Training data can be found here

In [2]:
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

print(train_data.shape)
print(test_data.shape)

(31962, 3)
(17197, 2)


In [3]:
train_data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test_data.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
#cleaning the dataset

import re
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", "",tweet.lower()).split())

processed_tweet=[]
for item in range(train_data.shape[0]):
  processed_tweet.append(process_tweet(train_data['tweet'][item]))

In [6]:
train_data['tweet'][9]

" @user @user welcome here !  i'm   it's so #gr8 ! "

In [7]:
train_data['processed_tweet']=processed_tweet

train_data.head()

,id,label,tweet,processed_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i cant use cause they d...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now motivation


In [10]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_data['processed_tweet'].values)
X = tokenizer.texts_to_sequences(train_data['processed_tweet'].values)
X = pad_sequences(X)

In [11]:
Y = pd.get_dummies(train_data['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(25569, 32) (25569, 2)
(6393, 32) (6393, 2)


In [13]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=200)

model.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [24]:
y_pred=np.argmax(model.predict(X_test), axis=-1)

In [28]:
Y_test=np.argmax(Y_test,axis=-1)

In [29]:
print(y_pred)
print(Y_test)

[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]


In [30]:
from sklearn.metrics import classification_report as cr

print(cr(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5937
           1       0.98      0.27      0.43       456

    accuracy                           0.95      6393
   macro avg       0.96      0.64      0.70      6393
weighted avg       0.95      0.95      0.93      6393



In [31]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(Y_test,y_pred)

array([[5934,    3],
       [ 332,  124]])

Deep Learning tools 

In [33]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,SpatialDropout1D,Dense

In [34]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 32, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_data['processed_tweet'].values)
X = tokenizer.texts_to_sequences(train_data['processed_tweet'].values)
X = pad_sequences(X)

In [40]:
Y = pd.get_dummies(train_data['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(25569, 32) (25569, 2)
(6393, 32) (6393, 2)


In [42]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2,validation_data=(X_test ,Y_test))

Epoch 1/10
800/800 - 113s - loss: 0.1122 - accuracy: 0.9606 - val_loss: 0.1475 - val_accuracy: 0.9542
Epoch 2/10
800/800 - 113s - loss: 0.0975 - accuracy: 0.9636 - val_loss: 0.1339 - val_accuracy: 0.9548
Epoch 3/10
800/800 - 113s - loss: 0.0868 - accuracy: 0.9677 - val_loss: 0.1465 - val_accuracy: 0.9548
Epoch 4/10
800/800 - 113s - loss: 0.0771 - accuracy: 0.9707 - val_loss: 0.1398 - val_accuracy: 0.9567
Epoch 5/10
800/800 - 112s - loss: 0.0669 - accuracy: 0.9746 - val_loss: 0.1655 - val_accuracy: 0.9465
Epoch 6/10
800/800 - 113s - loss: 0.0603 - accuracy: 0.9768 - val_loss: 0.1642 - val_accuracy: 0.9506
Epoch 7/10
800/800 - 113s - loss: 0.0517 - accuracy: 0.9803 - val_loss: 0.1835 - val_accuracy: 0.9509
Epoch 8/10
800/800 - 113s - loss: 0.0480 - accuracy: 0.9822 - val_loss: 0.1845 - val_accuracy: 0.9501
Epoch 9/10
800/800 - 113s - loss: 0.0409 - accuracy: 0.9848 - val_loss: 0.2212 - val_accuracy: 0.9504
Epoch 10/10
800/800 - 113s - loss: 0.0360 - accuracy: 0.9867 - val_loss: 0.2060 - 